## Finetuning Gemma2 for lease agreement data extraction

### Install dependencies

In [1]:
%pip install pandas peft scikit-learn transformers datasets torch trl accelerate bitsandbytes huggingface-hub -q -U

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

def messages_from_df(dataframe: pd.DataFrame) -> dict[str, list[dict[str, str]]]:
    # Transform the DataFrame into the desired format
    messages = []
    for _, row in dataframe.iterrows():
        message = [
            {
                "role": "user",
                "content": "Extract lease agreement data from the following OCR text:\n" + row['extracted_text']
            },
            {
                "role": "assistant",
                "content": row['extracted_fields']
            }
        ]
        messages.append(message)
    return {"messages": messages}

### Build the dataframe
Build the dataframe from the collected data

In [3]:

import sqlite3

# Path to the SQLite database
db_path = "../output/extracted_lease_agreements.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Query to select all data from the extracted_data table
query = "SELECT * FROM extracted_data"

# Read the data into a DataFrame
df = pd.read_sql_query(query, conn, index_col="id")

# Close the database connection
conn.close()

### Split the dataset

In [4]:
from sklearn.model_selection import train_test_split

# First split: 80% for training, 20% for temp (which will be split into eval and test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: 50% of temp for eval and 50% for test (10% of the original data each)
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Check the sizes of the splits
print(f"Training set size: {len(train_df)}")
print(f"Evaluation set size: {len(eval_df)}")
print(f"Test set size: {len(test_df)}")


Training set size: 205
Evaluation set size: 26
Test set size: 26


### Load the base model

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
import torch

# login to access gated model (gemma-2b-it)
notebook_login()

# Load the tokenizer and model
model_id = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)  # Change num_labels based on your task


`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Next, let's setup LORA config

In [6]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha = 32, # increased alpha
    lora_dropout=0.05, # decreased dropout
    r=32, # increased rank
    task_type='CAUSAL_LM'
)

### Build the datasets

In [7]:
from datasets import Dataset

# Create datasets with the extracted text and labels
train_dataset = Dataset.from_dict(messages_from_df(train_df))
eval_dataset = Dataset.from_dict(messages_from_df(eval_df))
test_dataset = Dataset.from_dict(messages_from_df(test_df))


### Train (fine tune) the base model

In [8]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Changed to evaluate at the end of each epoch
    learning_rate=3e-5,
    lr_scheduler_type="linear",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=4,
    num_train_epochs=4,  # Increase the number of epochs
    use_cpu=True,
    save_strategy="steps",  # Save checkpoints based on steps
    save_steps=100,  # Save a checkpoint every 100 steps
    save_total_limit=3,  # Limit the total number of checkpoints to 3
)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=512,
    args=training_args
)

trainer.train()

/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': neftune_noise_alpha, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/aryaniyaps/velocius-ai/leaseaudit-model-training/.venv/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/aryaniyaps/velocius

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
0,No log,3.101158
1,No log,2.865527
2,No log,2.794258
3,No log,2.777667


TrainOutput(global_step=100, training_loss=2.9492678833007813, metrics={'train_runtime': 29293.1559, 'train_samples_per_second': 0.028, 'train_steps_per_second': 0.003, 'total_flos': 4858961824382976.0, 'train_loss': 2.9492678833007813, 'epoch': 3.883495145631068})

### Save the model

In [9]:
trainer.model.save_pretrained("./saved_models/finetuned_gemma2_updated")

### Check model accuracy

In [10]:
from peft import PeftModelForCausalLM
from datasets import load_metric

finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="./saved_models/finetuned_gemma2_updated")

# Set the model to evaluation mode
finetuned_model.eval()

# Initialize BLEU metric
bleu_metric = load_metric("bleu", trust_remote_code=True)

# Prepare lists to store outputs and true labels
finetuned_outputs = []
normal_outputs = []
true_labels = []

# Disable gradient calculation for evaluation
with torch.no_grad():
    for message in test_dataset['messages']:
        # Prepare input using apply_chat_template
        input_ids = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=True, return_tensors="pt").to("cuda")
        print("Input IDs generated")
        # Get the true output
        true_output = message[1]['content']
        print("True output: ", true_output)
        true_labels.append(true_output)

        # Generate output from the fine-tuned model
        outputs_finetuned = finetuned_model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)
        finetuned_output = tokenizer.decode(outputs_finetuned[0]).split('<start_of_turn>model\n')[-1]
        print("Finetuned output: ", finetuned_output)
        finetuned_outputs.append(finetuned_output)

        # Generate output from the normal model
        outputs_normal = model.generate(input_ids=input_ids, max_new_tokens=1024, do_sample=False)
        normal_output = tokenizer.decode(outputs_normal[0]).split('<start_of_turn>model\n')[-1]
        print("Normal output: ", normal_output)
        normal_outputs.append(normal_output)

# Calculate BLEU scores
bleu_score_finetuned = bleu_metric.compute(predictions=finetuned_outputs, references=[[true] for true in true_labels])
bleu_score_normal = bleu_metric.compute(predictions=normal_outputs, references=[[true] for true in true_labels])

# Print BLEU scores
print(f"Finetuned Model BLEU Score: {bleu_score_finetuned['bleu']:.4f}")
print(f"Normal Model BLEU Score: {bleu_score_normal['bleu']:.4f}")

# Print generated outputs for comparison
for finetuned_output, normal_output, true_output in zip(finetuned_outputs, normal_outputs, true_labels):
    print("Finetuned: " + finetuned_output)
    print("Normal   : " + normal_output)
    print("True     : " + true_output)
    print("-" * 40)

/tmp/ipykernel_24627/1719675052.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu", trust_remote_code=True)


Input IDs generated
True output:  {'tenant_name': None, 'unit_address': '333 H Street, Ste. 5000, Chula Vista, CA 91910', 'unit_number': None, 'unit_type': None, 'agreement_date': None, 'lease_start': None, 'lease_end': '2024-03-31', 'lease_auto_renew': None, 'hourly_rate': None, 'monthly_rent': 3350.0, 'prorated_rent': 2121.67, 'security_deposit': 2500.0, 'lease_rent': None, 'monthly_payment_breakdown': {'Liability to Landlord Insurance': 9.5, 'Admin Fee - Liability to Landlord Insurance': 3.0, 'Rent Income': 3350.0, 'Total': 3362.5}, 'utility_charges': None}
Finetuned output:  This context does not provide any lease agreement data, so I cannot extract the requested information from the context.<eos>
Normal output:  This context does not provide any lease agreement data, so I cannot extract the requested information from the context.<eos>
Input IDs generated
True output:  {'tenant_name': 'Saray Ramos Gutierrez', 'unit_address': 'Crystal Terrace', 'unit_number': None, 'unit_type': None

KeyboardInterrupt: 